---
# 프로젝트 명: MLB 데이터를 활용한 회귀모델 생성 및 결과분석

## 데이터 출처: [Moneyball | Kaggle](https://www.kaggle.com/wduckett/moneyball-mlb-stats-19622012)

## 프로젝트 목표
    MLB Moneyball 데이터와 강의 실습시간에 배운 내용으로 회귀분석 및 로지스틱회귀분석 모델 생성
        - 한 시즌 동안 승리한 횟수(W) 예측 회귀분석 모델, 플레이오프 진출 여부(Playoffs) 결정 로지스틱회귀분석 모델 생성
    독립변수들과 종속변수와의 인과관계를 고려하여 모델에 영향력이 큰 유의미한 독립변수 찾기
        - 기존의 독립변수를 조합하여 만든 변수로 예측해보기
        - 변수선택법(전진선택법, 후진소거법)으로 최적의 변수 조합 찾기
        - 다중공선성 문제 확인
    회귀모델의 결과를 해석하는 방법 습득

## 프로젝트 구성
    - 시각화를 통한 데이터의 이해
    - RS를 독립변수로 W를 예측하는 단순선형회귀 모델 생성
    - (RS-RA)를 독립변수로 W를 예측하는 단순선형회귀 모델 생성
    - 회귀분석 결과의 해석
    - 모든 변수를 활용한 다중회귀분석 및 다중공선성 문제
    - 로지스틱회귀 모델 생성
    - 변수 선택법으로 로지스틱회귀분석 정확도 올리기

## 프로젝트 과정
    - 데이터의 간단한 시각화에서부터 회귀분석과 로지스틱회귀분석 문제 해결까지 강의 실습 내용 확인
    - 모델 생성 및 해석에 대한 내용에 집중하기 위해서 학습데이터와 테스트데이터를 구분하지 않고 진행
    - 강의 실습 시간에 다룬 자료를 이용해서 코드 작성

- 작성자: 김민중 감수자

---

### Context

In the early 2000s, Billy Beane and Paul DePodesta worked for the Oakland Athletics. While there, they literally changed the game of baseball. They didn't do it using a bat or glove, and they certainly didn't do it by throwing money at the issue; in fact, money was the issue. They didn't have enough of it, but they were still expected to keep up with teams that had much deeper pockets. This is where Statistics came riding down the hillside on a white horse to save the day. This data set contains some of the information that was available to Beane and DePodesta in the early 2000s, and it can be used to better understand their methods.

### Content

This data set contains a set of variables that Beane and DePodesta focused heavily on. They determined that stats like on-base percentage (OBP) and slugging percentage (SLG) were very important when it came to scoring runs, however they were largely undervalued by most scouts at the time. This translated to a gold mine for Beane and DePodesta. Since these players weren't being looked at by other teams, they could recruit these players on a small budget. The variables are as follows:

- Team, 팀
- League, 리그
- Year, 연도
- Runs Scored (RS), 득점 수
- Runs Allowed (RA), 실점 수
- Wins (W), 승리 횟수
- On-Base Percentage (OBP), 출루율
- Slugging Percentage (SLG), 장타율
- Batting Average (BA), 타율
- Playoffs (binary), 플레이오프 진출 여부
- RankSeason, 시즌 순위
- RankPlayoffs 플레이오프 순위
- Games Played (G), 경기 수
- Opponent On-Base Percentage (OOBP), 도루 허용률
- Opponent Slugging Percentage (OSLG), 피장타율

### [Baseball statistics](https://en.wikipedia.org/wiki/Baseball_statistics)을 살펴보면 각 변수의 정확한 의미를 파악할 수 있다.

In [0]:
# 참고: 프로젝트 출제자의 Python 및 주요 라이브러리 버전

import pandas as pd 
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import sys

print("python version: ", sys.version)
print("pandas version: ", pd.__version__)
print("statsmodels version: ", sm.__version__)
print("numpy version: ", np.__version__)

%matplotlib inline

In [0]:
# Kaggle의 정책상 프로젝트 참여자는 Kaggle에 직접 로그인해서 자료를 다운로드하는 것을 권합니다.
# moneyball = pd.read_csv("https://mjgim-fc.s3.ap-northeast-2.amazonaws.com/baseball.csv", encoding="utf8")

# 데이터 불러오기(자료는 data 폴더에 있음)
moneyball = pd.read_csv("./data/baseball.csv", encoding="utf8")
moneyball.head()

In [0]:
# 데이터의 간단한 정보 파악(na의 개수 및 데이터 타입)
moneyball.info()

In [0]:
# na가 있는 컬럼인 RankSeason, RankPlayoffs을 제거(프로젝트에서 사용 안 함)
moneyball = moneyball.dropna(axis=1)
moneyball.info()

# 1232개의 object와 11개의 변수

In [0]:
# Object 변수 및 불필요한 변수 제거해서 단순 시각화

selected_df = moneyball.select_dtypes(exclude=['object'])
selected_df = selected_df.drop(["Year"], axis=1)
_ = pd.plotting.scatter_matrix(selected_df,
                               alpha = 0.5, figsize=(16,12))

## 점검 및 합격 기준표(패캠 전달 내용)
- 단순선형회귀 및 로지스틱회귀분석 모델의 계수는 필수적으로 구해야 함(동일한 실습 코드로 결과 도출 가능)
- 각각 서술식 질문에 대해서 의도를 제대로 이해하고 응답했는지를 판단
- 확실하다고 말씀드리기는 어려우나 강의 실습코드의 전진선택법과 후진소거법 코드가 잘못됐을 가능성이 있음(전진선택법 후진소거법은 평가에 반영 안하는게 좋을 것 같습니다.) 또한, VIF 계산할 때 상수항을 넣어주어야 하는데 실습코드에 상수항이 있는 경우도 있고 없는 경우도 있습니다.

## STEP 1. 시각화를 통한 데이터의 이해 
- 다양한 수치값을 갖는 변수들의 산포도를 보고 받은 통찰(insight)은 무엇인가?
- 강한 상관관계를 보이는 데이터들은 존재하는가?
- 경기 수(G)의 히스토그램은 어떤 의미를 갖는가?

### 답안 예시
- RS는 총 득점 수로 공격력과 관련된 지표인 OBP, SLG, BA와 강한 선형 관계를 보임. 
- 실점 수 RA는 W와 약한 음의 관계를 보임.
- 경기 수(G)는 다른 변수들과 연관성이 나타나지 않고 162 근처에 데이터들이 몰려 있음.

## STEP 2. RS를 독립변수로 W를 예측하는 단순선형회귀 모델 생성
- 가정: 시즌 총 득점(RS)이 승리에 영향을 줄 것이다.
- RS를 독립변수로 W를 예측한 단순선형모델 직선의 기울기 $\alpha$와 절편 $\beta$는 몇인가?

$$
W = \alpha RS + \beta
$$


- 해당 모델이 얼마나 적합한지를 평가하는 $R^2$는 몇 인가?
- 적당한 모델이라고 할 수 있는가?

In [0]:
moneyball.plot.scatter(x = "RS", y = "W", alpha = 0.7)

### 답안 예시
- 기울기: 0.0641, 절편: 35.0964, 즉 다음과 같이 선형 직선식으로 표현 가능.

$$
W = 0.0641 RS + 35.0964
$$

- R squared 값은 0.262로 만족하기 어려움.
- 적당한 모델이라고 할 수 없음

In [0]:
# sm으로 fit한 모델명은 fit_simple_model 으로 하시오.

# 해당 셀 비공개
target = moneyball["W"]

RS =  moneyball["RS"]
RS1 = sm.add_constant(RS, has_constant='add')

simple_model = sm.OLS(target, RS1)
fit_simple_model = simple_model.fit()
fit_simple_model.summary()

# fit_simple_model.params

In [0]:
# 참고
# Fit된 직선 그리기. sm을 이용해서 선형회귀분석을 한 경우 
# 아래와 같이 간단한 코드로 적합된 직선과 원래 데이터의 그림을 그릴 수 있음.

fig, ax = plt.subplots()
fig = sm.graphics.plot_fit(fit_simple_model, 1, ax=ax)
ax.set_ylabel("W")
ax.set_xlabel("RS")
ax.set_title("Linear Regression")

### Comments
- 득점 RS로 W를 예측한 단순선형회귀분석의 적합도는 만족하기 어려움(R squared 값으로 판단).
- 시즌 총 실점 RA를 독립변수로 W를 예측하는 단순선형회귀분석의 결과는 어떠한가? 만족할 만한가? 
- RS를 RA로 변경해서 직접 R squared를 구해보시오. 

### 답안 예시
- RA를 독립변수로 W를 예측하는 단순선형회귀분석 결과 R squared 값은 0.283로 RS ~ W 모델과 마찬가지고 만족할 수 없음

## 2. (RS-RA)를 독립변수로 W를 예측하는 단순선형회귀 모델 생성
- 득점과 실점이 승리 수와 관련이 없을까? 경기에서 승리하려면 상대보다 득점을 많이 해야한다. 즉, (득점 - 실점)을 새로운 독립변수로 설정하고 W를 예측하는 단순선형회귀분석을 해보시오.
- 가정: 시즌 총 득점(RS)과 실점(RA)의 차이가 승리에 영향을 줄 것이다.
- 강의 실습 시간에 학습한 statsmodels 라이브러리를 이용해서 아래의 질문에 답하시오.
- (RS-RA)를 독립변수로 W를 예측한 단순선형모델의 기울기 $\alpha$와 절편 $\beta$는 몇인가?

$$
W = \alpha \cdot (RS-RA) + \beta
$$

- 해당 모델이 얼마나 적합한지를 평가하는 $R^2$는 몇 인가?
- 적당한 모델이라고 할 수 있는가?

### 답안 예시
- 기울기: 0.1045, 절편: 80.9042, 즉 다음과 같이 선형 직선식으로 표현 가능.

$$
W = 0.1045 (RS-RA) + 80.9042
$$

- R squared 값은 0.880로 만족스러운 결과를 얻음
- 적당한 모델이라고 할 수 있음

In [0]:
# 해당 셀 비공개

target = moneyball["W"]

moneyball["RSmRA"] =  moneyball["RS"] - moneyball["RA"]
RSmRA = moneyball["RSmRA"]
RSmRA1 = sm.add_constant(RSmRA, has_constant='add')

simple_model2 = sm.OLS(target, RSmRA1)
fit_simple_model2 = simple_model2.fit()
fit_simple_model2.summary()

## 3. 회귀분석 결과의 해석
- 득점과 실점이 개별적으로 한 개씩만 본다면 승리 예측에 큰 영향을 주지 못하지만 (득점 - 실점)으로 결합한 변수는 승리 예측에 유의미하게 영향을 주었다. 이런 작용은 무엇이라고 하는가?
- 다시 RS, RA를 두 개의 독립변수로 W를 예측한 선형모델의 기울기 $\alpha_1$, $\alpha_2$와 절편 $\beta$는 몇인가?

$$
W = \alpha_1 RS + \alpha_2 RA + \beta
$$

- RS, RA를 두 개의 독립변수로 W를 예측한 모델과 (RS-RA)을 독립변수로 W를 예측한 모델은 무슨 차이가 있을까?
- 회귀분석 결과인 $\alpha_1$, $\alpha_2$와 (RS-RA)의 계수인 $\alpha$를 비교해보고 RS와 RA의 차이가 승리에 큰 영향을 미친다고 결론 내릴 수 있는가?

### 답안 예시
- RS ~ W, RA ~ W 선형 모델 모두 만족스러운 결과를 얻지 못했지만 (RS-RA) ~ W 모델은 만족스러운 결과를 주었다. 즉, (RS-RA)는 W와 강한 선형 관계에 있다고 말할 수 있다. 개별 변수는 W에 큰 영향을 주지 못했지만 두 변수의 차이는 큰 영향을 주었는데 이러한 작용은 **교호작용**이라고 한다.

- $\alpha_1:$ 0.1045, $\alpha_2:$ -0.1046, $\beta:$ 80.9805, 즉 다음과 같이 표현 가능.

$$
W = 0.1045 RS - 0.1046RA  + 80.9805
$$

- (RS-RA) ~ W 모델은 (RS-RA)에 곱해지는 기울기와 절편, 즉 두 개의 계수를 찾는 모델이다. 그리고 RS, RA 두 독립변수로 W를 예측하는 모델은 RS와 RA에 각각 곱해지는 계수들과 절편, 즉 세 개의 계수를 찾는 모델이다. 이 모델은 (RS-RA) ~ W 모델과는 다르게 RS와 RA에 곱해지는 계수들은 독립적으로 결정된다.
- 그렇다. 독립적으로 얻은 $\alpha_1$과 $\alpha_2$ 값의 조합이 (RS-RA)의 계수인 $\alpha$와 매우 가까운 값으로 나왔다. 즉 (RS-RA)가 승리 수 W 예측에 큰 영향을 준다고 할 수 있다.

In [0]:
# 해당 셀 비공개

RSRA = moneyball[["RS", "RA"]]
RSRA1 = sm.add_constant(RSRA, has_constant='add')

multi_model = sm.OLS(target, RSRA1)
multi_model = multi_model.fit()
multi_model.summary()

## 4. 모든 변수를 활용한 다중회귀분석 및 다중공선성 문제
- [RS, RA, OBP, SLG, BA, G] 6개 독립변수로 W를 예측하는 다중회귀분석 하시오.
- RS, RA 두 독립변수의 모델과 비교했을 때 결과가 향상됐다고 할 수 있는가?
- 결과가 향상되지 않았다면 다중공선성 문제가 있을 수 있다고 판단할 수 있는데 정량적인 수치로 확인해보시오. VIF를 구하고 결과를 해석하시오(특별히 RS의 VIF 수치가 높게 나온 이유를 한번 생각해 보시오).
    - RS, RA 두 변수로도 충분히 W를 설명 가능했었습니다. 이를 어떻게 해석해야 할까요?
    - G의 VIF 수치는 어떻게 해석해야 할까요? 가장 낮은 VIF가 예측에 중요한 변수인가요?
- 주의 statsmodels에는 항상 상수항을 넣어주어야 한다. VIF도 마찬가지임. [참고](https://stackoverflow.com/questions/42658379/variance-inflation-factor-in-python)
- 강의 실습 때 다룬 전진선택법을 사용해서 AIC가 가장 작은 값이 나오는 독립변수를 찾으시오.

### 답안 예시
- 아래와 같이 코드 작성.
- 향상됐다고 할 수 없다. R squared 값이 향상되지 않았다. 즉, 굳이 OBP, SLG, BA, G 변수를 사용할 필요 없다.
- 아래와 같이 VIF 코드 작성(VIF 계산 시 위와 같이 상수항을 추가해야 함.).
- 데이터 시각화에서 본 것과 같이 총 득점 수 RS와 나머지 공격에 관련된 변수인 OBP, SLG, BA 등으로 인해서 강한 다중공선성 문제가 있어보임. 즉, 모든 변수를 사용해서 예측할 필요성이 없다고 보임.
- G의 VIF 수치가 가장 낮다. 즉 다른 변수와의 선형 연관성은 없어 보인다. 그 이상의 이하도 아님. 예측에 중요한 변수인지 아닌지 판단할 수 없다.
- 강사님 전진선택법 코드가 버그? 에러가 있어보입니다. 확실하지는 않지만 그래서 전진선택법이 중요하긴 한데 평가에서 빼도 좋을 것 같습니다.

In [0]:
# 해당 셀 비공개

x_data = moneyball[["RS", "RA", "OBP", "SLG", "BA", "G"]]
x_data1 = sm.add_constant(x_data, has_constant='add')

multi_model = sm.OLS(target, x_data1)
multi_model = multi_model.fit()
multi_model.summary()

In [0]:
# 해당 셀 비공개

from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    x_data1.values, i) for i in range(x_data1.shape[1])]
vif["features"] = x_data1.columns
vif.iloc[1: -1]

In [0]:
# 참고: 경기수 G 히스토그램
moneyball["G"].value_counts().plot.barh()
# 대부분 162 근처에 있음

In [0]:
# 참고로 선형회귀분석의 다양한 통계적인 결과 수치를 원치 않는 경우
# sklearn을 이용해서 손쉽게 회귀분석 모델을 만들 수 있다.

from sklearn.linear_model import LinearRegression

# Fit 할 때 상수항을 따로 추가할 필요 없음
reg = LinearRegression().fit(moneyball[["RS", "RA"]], moneyball["W"])
print("r squared:", reg.score(moneyball[["RS", "RA"]], moneyball["W"]))
print("coefficients: ", reg.coef_)
print("intercept: ", reg.intercept_)

## 5. 로지스틱회귀 모델 생성
- RS와 RA 두 독립변수로 플레이오프(Playoffs) 진출 여부를 결정하는 로지스틱회귀분석 모델을 생성해보시오.
- Confusion matrix, AUC, ROC 곡선을 이용해서 결과를 해석해 보시오.
- 참고
    - [Scikit learn confusion matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)
    - [Model Evaluation Techniques for Classification models](https://towardsdatascience.com/model-evaluation-techniques-for-classification-models-eac30092c38b)
    - [Evaluating a Classification Model]()

### 답안 예시
- 아래와 같이 코드 작성. p-value 모두 0.000 로 RS, RA 변수 모두 플레이오프 예측에 유의미한 결과를 얻었다.
- Confusion matrix, AUC, ROC 곡선도 아래와 같이 코드로 계산. ROC 커브를 보면 해당 로지스틱회귀분석 모델은 만족할 만함.

In [0]:
# 해당 셀 비공개

model = sm.Logit(moneyball["Playoffs"], RSRA1)
fit_logit = model.fit()

fit_logit.summary()

In [0]:
# 참고로 선형회귀분석과 마찬가지로 sklearn을 이용해서 손쉽게 회귀분석 모델을 만들 수 있다.
# 위 결과와 비교해보시오.

from sklearn.linear_model import LogisticRegression

# Fit 할 때 상수항을 따로 추가할 필요 없음
reg = LogisticRegression().fit(moneyball[["RS", "RA"]], moneyball["Playoffs"])
print("mean accuracy:", reg.score(moneyball[["RS", "RA"]], moneyball["Playoffs"]))
print("coefficients: ", reg.coef_)
print("intercept: ", reg.intercept_)

### Comments
- 로지스틱회귀분석은 주어진 독립변수의 공간을 선형으로 분리한다고 수학적으로 해석할 수 있다. 조금 더 자세히 설명하면 로지스틱회귀분석의 결과로 우리는 아래 식의 계수들 $\alpha_1, \alpha_2, \beta$를 얻었다. 플레이오프의 진출 여부를 0 ~ 1사이의 확률 값으로 출력하는 식이 로지스틱회귀분석의 결과이다. 여기서 진출 여부의 판단 기준을 $1/2$로 한다고 가정해보자(일반적으로 $1/2$ 확률로 판단하지만 모델이나 상황에 따라서 조절할 수도 있다).

$$
\text{Playoffs} = \frac{1}{1 + e^{(\alpha_1 RS + \alpha_2 RA + \beta)}}
$$

- 왼쪽 플레이오프 진출 확률을 1/2로 두고 식을 정리하면 다음과 같다.

$$
\alpha_1 RS + \alpha_2 RA + \beta = 0
$$

- 결과적으로 위 식을 만족하는 RS와 RA 값들은 아래와 같은 빨간색 선과 같은 직선의 형태를 띤다. 직선(선형)을 기준으로 한쪽 영역은 플레이오프 진출 못함, 반대쪽은 플레이오프 진출함으로 판단되는 것이다(물론 로지스틱회귀분석모델의 판단). 

![](https://mjgim-fc.s3.ap-northeast-2.amazonaws.com/logistic.png)

- 이처럼 로지스틱회귀분석은 주어진 독립변수 공간을 선형으로 분리해서 이진 분류하는 방식이기 때문에 XOR 문제를 해결하기 어려운 것이다(XOR은 선형으로 해결 불가능). 그림 출처: https://web.stanford.edu/~jurafsky/slp3/7.pdf

![](https://mjgim-fc.s3.ap-northeast-2.amazonaws.com/xor.png)

In [0]:
line_x = moneyball["RS"]

line_y = (- fit_logit.params["RS"] * moneyball["RS"]  - fit_logit.params["const"] ) \
          / fit_logit.params["RA"]

moneyball.plot.scatter(x = "RS", y = "RA", alpha = 0.5)
plt.plot(line_x, line_y, "r")

In [0]:
# 해당 셀 비공개

from sklearn.metrics import confusion_matrix
from sklearn import metrics

prediction = (fit_logit.predict(RSRA1) >= 0.5) * 1
confusion_matrix(moneyball["Playoffs"], prediction)

In [0]:
# 해당 셀 비공개

fpr, tpr, thresholds = metrics.roc_curve(moneyball["Playoffs"], 
                                         fit_logit.predict(RSRA1), pos_label=1)

# Print ROC curve
plt.plot(fpr,tpr)
plt.plot([0,1], [0,1])
# Print AUC
auc = np.trapz(tpr,fpr)
print('AUC:', auc)

## 6. 변수 선택법으로 로지스틱회귀분석 정확도 올리기
- RS, RA, OBP, SLG, BA, G을 독립변수를 사용해서(상수항 포함) 플레이오프 진출 여부를 결정하는 로지스틱회귀분석 모델을 만들어 보시오.
- 후진소거법으로 최적의 독립변수를 찾아 보시오(AIC 값이 크게).

### 답안 예시
- 아래와 같이 코드 작성.
- 후진소거법은 평가에서 제외.

In [0]:
# 해당 셀 비공개

model = sm.Logit(moneyball["Playoffs"], x_data1)
fit_multi_logit = model.fit()

fit_multi_logit.summary()

In [0]:
X = x_data1
y = moneyball["Playoffs"]
    
import itertools

def processSubset(X,y, feature_set):
            model = sm.Logit(y,X[list(feature_set)])
            regr = model.fit()
            AIC = regr.aic
            return {"model":regr, "AIC":AIC}
        
        
def backward(X,y,predictors):
    results = []
    
    # 데이터 변수들이 미리정의된 predictors 조합 확인
    for combo in itertools.combinations(predictors, len(predictors) - 1):
        results.append(processSubset(X=X, y= y,feature_set=list(combo)+['const']))
    models = pd.DataFrame(results)
    
    # 가장 낮은 AIC를 가진 모델을 선택
    best_model = models.loc[models['AIC'].argmin()]
    print("Processed ", models.shape[0], "models on", len(predictors) - 1, "predictors in")
    print('Selected predictors:',best_model['model'].model.exog_names,' AIC:',best_model[0] )
    return best_model


def backward_model(X, y):
    Bmodels = pd.DataFrame(columns=["AIC", "model"], index = range(1,len(X.columns)))
    predictors = X.columns.difference(['const'])
    Bmodel_before = processSubset(X,y,predictors)['AIC']
    while (len(predictors) > 1):
        Backward_result = backward(X=X, y= y, predictors = predictors)
        if Backward_result['AIC'] > Bmodel_before:
            break
        Bmodels.loc[len(predictors) - 1] = Backward_result
        predictors = Bmodels.loc[len(predictors) - 1]["model"].model.exog_names
        Bmodel_before = Backward_result['AIC']
        predictors = [ k for k in predictors if k != 'const']

    return (Bmodels['model'].dropna().iloc[0])



In [0]:
Backward_best_model = backward_model(X=x_data1, y=moneyball["Playoffs"])

## 참고
- 통계 라이브러리에 특화(?)된 R에서는 회귀분석 모델 생성, 변수선택법, VIF 등 통계분석을 쉽게 수행할 수 있다.
- 위에서 수행했던 내용을 R로 실행한 내용입니다.
- [R vs Python 회귀분석모델](https://s3.ap-northeast-2.amazonaws.com/mjgim.assets/2.+Regression+in+R.html)